In [1]:
open('tarantino.csv')

<_io.TextIOWrapper name='tarantino.csv' mode='r' encoding='UTF-8'>

In [56]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

In [100]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [5]:
curse = pd.read_csv('tarantino.csv', encoding="UTF-8")
curse.sample(5)

,movie,type,word,minutes_in
570,Pulp Fiction,word,fuck,54.67
1691,Jackie Brown,word,motherfucking,51.97
344,Reservoir Dogs,word,bitch,77.06
500,Pulp Fiction,word,motherfucker,19.57
1852,Jackie Brown,word,motherfucking,131.00


In [10]:
curse.sample(20)

,movie,type,word,minutes_in
483,Pulp Fiction,word,motherfucker,12.95
1737,Jackie Brown,word,ass,76.92
126,Reservoir Dogs,word,fucking,28.83
1487,Django Unchained,word,motherfucker,152.35
1185,Inglorious Basterds,word,fuck,121.63
336,Reservoir Dogs,word,bitch,74.40
201,Reservoir Dogs,word,asshead,42.86
633,Pulp Fiction,word,n-word,70.55
1827,Jackie Brown,word,ass,124.68
787,Pulp Fiction,word,motherfucking,121.03


In [6]:
type(curse.minutes_in)

pandas.core.series.Series

In [8]:
curse.dtypes

movie          object
type           object
word           object
minutes_in    float64
dtype: object

In [160]:
curse.movie.unique()

array(['Reservoir Dogs', 'Pulp Fiction', 'Kill Bill: Vol. 1',
       'Kill Bill: Vol. 2', 'Inglorious Basterds', 'Django Unchained',
       'Jackie Brown'], dtype=object)

In [11]:
open('credits.csv')

<_io.TextIOWrapper name='credits.csv' mode='r' encoding='UTF-8'>

In [12]:
credits=pd.read_csv('credits.csv', encoding="UTF-8")
credits.sample(5)

,movie_id,title,cast,crew
4166,157293,ABCD (Any Body Can Dance),"[{""cast_id"": 7, ""character"": ""Vishnu"", ""credit...","[{""credit_id"": ""52fe4bbd9251416c910e4437"", ""de..."
3692,10822,Savage Grace,"[{""cast_id"": 3, ""character"": ""Barbara Baekelan...","[{""credit_id"": ""52fe43bd9251416c7501c39f"", ""de..."
797,24438,Did You Hear About the Morgans?,"[{""cast_id"": 1, ""character"": ""Paul Morgan"", ""c...","[{""credit_id"": ""5776cb279251414643001333"", ""de..."
643,5551,Space Cowboys,"[{""cast_id"": 11, ""character"": ""Frank Corvin"", ...","[{""credit_id"": ""52fe440ec3a36847f807fc47"", ""de..."
3473,15256,200 Cigarettes,"[{""cast_id"": 1, ""character"": ""Bartender"", ""cre...","[{""credit_id"": ""52fe46479251416c75073207"", ""de..."


In [14]:
credits.cast.sample(5)

50      [{"cast_id": 5, "character": "Prince Dastan", ...
2301    [{"cast_id": 1, "character": "Matt Franklin", ...
2950    [{"cast_id": 3, "character": "Dorita Evita Per...
977     [{"cast_id": 18, "character": "Hogarth Hughes ...
560     [{"cast_id": 1, "character": "Jimmy Bly", "cre...
Name: cast, dtype: object

In [15]:
open('movies.csv')

<_io.TextIOWrapper name='movies.csv' mode='r' encoding='UTF-8'>

In [17]:
movies=pd.read_csv('movies.csv', encoding="UTF-8")
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2782,10000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",http://www.inthelandofbloodandhoney.com/,79777,"[{""id"": 789, ""name"": ""muslim""}, {""id"": 1156, ""...",en,In the Land of Blood and Honey,"During the Bosnian War, Danijel, a soldier fig...",4.267466,"[{""name"": ""FilmDistrict"", ""id"": 7263}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-12-23,303877,127.0,"[{""iso_639_1"": ""bs"", ""name"": ""Bosanski""}, {""is...",Released,Love can change what we want… war can change w...,In the Land of Blood and Honey,5.8,45
4696,8000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",http://weekenderfilm.tumblr.com/,79120,"[{""id"": 237, ""name"": ""gay""}, {""id"": 1025, ""nam...",en,Weekend,After a drunken house party with his straight ...,1.041254,"[{""name"": ""EM Media"", ""id"": 1917}, {""name"": ""T...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2011-09-22,469947,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A (sort of) love story between two guys over a...,Weekend,7.4,163
1319,38000000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",http://www.riddick-movie.com,87421,"[{""id"": 4565, ""name"": ""dystopia""}, {""id"": 9748...",en,Riddick,Betrayed by his own kind and left for dead on ...,46.630062,"[{""name"": ""One Race Productions"", ""id"": 1225},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-09-02,98337295,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Survival Is His Revenge,Riddick,6.2,2035
4718,120000,"[{""id"": 35, ""name"": ""Comedy""}]",http://www.adogsbreakfastmovie.com/,18632,"[{""id"": 10183, ""name"": ""independent film""}]",en,A Dog's Breakfast,If you've never been good at anything in your ...,2.213889,"[{""name"": ""Kibble Productions"", ""id"": 61307}]","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}]",2007-03-03,0,88.0,[],Released,Man's best friend makes the worst accomplice.,A Dog's Breakfast,5.9,24
4122,13500000,"[{""id"": 18, ""name"": ""Drama""}]",https://twitter.com/Stonewall_Movie,273899,"[{""id"": 173510, ""name"": ""stonewall riot""}]",en,Stonewall,"""Stonewall"" is a drama about a young man in Ne...",5.160230,"[{""name"": ""Centropolis Entertainment"", ""id"": 3...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-09-18,187674,129.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Where Pride Began,Stonewall,5.2,32


In [18]:
open('pfscript.csv')

<_io.TextIOWrapper name='pfscript.csv' mode='r' encoding='UTF-8'>

In [19]:
script=pd.read_csv('pfscript.csv', encoding="UTF-8")
script.sample(5)

,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count
506,507,Vincent,Vincent,False,False,living room,night,You're gonna give her the shot.,6
430,431,Vincent,Vincent,False,False,int. bathroom (Marsellus Wallace's house),night,"...It's a moral test of yourself, whether or n...",24
776,777,Maynard,Maynard,False,False,int. Mason-Dixie pawnshop,day,"Now you on top, stand up and come to the counter.",11
381,382,Vincent,Vincent,False,False,int. Jackrabbit Slim's (dining area),night,Is that a fact?,4
774,775,Butch,Butch,False,False,int. Mason-Dixie pawnshop,day,"Look mister, this ain't any of your business –",8


In [20]:
open('tmdb.csv')

<_io.TextIOWrapper name='tmdb.csv' mode='r' encoding='UTF-8'>

In [22]:
tmdb=pd.read_csv('tmdb.csv', encoding="UTF-8")
tmdb.sample(5)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
3662,77499,tt2091240,0.370236,0,0,Barbie: A Perfect Christmas,Diana Kaarina|Jennifer Waris|Rachel Harrison|N...,NaN,Terry Klassen|Mark Baldo,NaN,...,"Join Barbie and her sisters Skipper, Stacie an...",74,Animation|Family,NaN,7/1/2011,21,5.8,2011,0.00,0.000000e+00
1347,11590,tt0076723,0.407317,0,28000000,Slap Shot,Paul Newman|Strother Martin|Michael Ontkean|Je...,NaN,George Roy Hill,Slap Shot out slaps... out swears... out laugh...,...,"Located in the US Rust Belt, Charlestown is ho...",123,Drama|Comedy,Universal Pictures,2/25/1977,49,7.3,1977,0.00,1.007379e+08
6677,326,tt0417148,0.646745,33000000,62022014,Snakes on a Plane,Samuel L. Jackson|Julianna Margulies|Kenan Tho...,NaN,David R. Ellis,"At 30,000 feet, snakes aren't the deadliest th...",...,America is on the search for the murderer Eddi...,105,Horror|Action|Crime|Thriller,New Line Cinema,8/6/2006,236,5.2,2006,35695082.22,6.708730e+07
3487,83201,tt2268617,0.910291,0,0,Puss in Boots: The Three Diablos,Antonio Banderas|Gilles Marini|Chris Miller|Wa...,NaN,Raman Hui,An All-New Adventure!,...,Puss in Boots is on a mission to recover the P...,13,Animation|Family,NaN,12/8/2011,73,6.8,2011,0.00,0.000000e+00
1255,298935,tt3214248,0.076348,0,0,All Relative,Connie Nielsen|Jonathan Sadowski|Sara Paxton|A...,NaN,J.C. Khoury,NaN,...,Things couldn't be going better for Harry and ...,85,Romance|Comedy|Drama,Shoot First Entertainment,11/21/2014,11,5.7,2014,0.00,0.000000e+00


In [23]:
script.sample()

,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count
954,955,Jimmie,Jimmie,False,False,int. Jimmie's house,morning,Uh-huh.,1


In [38]:
script.head(15)

,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count
0,1,Young man,Pumpkin,False,False,int. coffee shop,morning,"No, forget it, it's too risky. I'm through doi...",11
1,2,Young woman,Honey Bunny,False,False,int. coffee shop,morning,"You always say that, the same thing every time...",15
2,3,Young man,Pumpkin,False,False,int. coffee shop,morning,I know that's what I always say. I'm always ri...,12
3,4,Young woman,Honey Bunny,False,False,int. coffee shop,morning,– but you forget about it in a day or two -,10
4,5,Young man,Pumpkin,False,False,int. coffee shop,morning,"– yeah, well, the days of me forgittin' are ov...",18
5,6,Young woman,Honey Bunny,False,False,int. coffee shop,morning,"When you go on like this, you know what you so...",12
6,7,Young man,Pumpkin,False,False,int. coffee shop,morning,"I sound like a sensible fucking man, is what I...",12
7,8,Young woman,Honey Bunny,False,False,int. coffee shop,morning,"You sound like a duck. Quack, quack, quack, qu...",12
8,9,Young man,Pumpkin,False,False,int. coffee shop,morning,"Well take heart, 'cause you're never gonna haf...",34
9,10,Young woman,Honey Bunny,False,False,int. coffee shop,morning,After tonight.,2


In [133]:
script['Character (actual)'] = script['Character (actual)'].replace('The Wolf','Wolf')


In [27]:
characters=set(script["Character (actual)"])
len(characters)

37

In [28]:
print(characters)

{'Waitress', 'Trudi', 'Zed', 'Maynard', 'Pedestrian', 'Looky-loo woman', 'English Dave', 'Lance', 'Jules', 'Sportscaster #1', 'Esmarelda', 'Roger', 'Marsellus', 'Butch', 'Klondike', 'Brett', 'Fabienne', 'Fourth man', 'Sportscaster #2', 'Mother', 'Buddy', 'Raquel', 'Preacher (Emil Simkus)', 'Vincent', 'Capt. Koons', 'Ed Sullivan', 'Honey Bunny', 'Gawker #2', 'Jimmie', 'Mia', 'Pumpkin', 'Jody', 'Gawker #1', 'The Gimp', 'Manager', 'Marvin', 'The Wolf'}


In [148]:
women=("Waitress", "Trudi", "Looky-loo woman", "Esmarelda", "Mother", "Raquel", "Honey Bunny", "Mia", "Jody", "Fabienne")

In [151]:

women_script = script[script['Character (actual)'].isin(women)]
women_indices = []
for i in range(len(women_script)-1):
    if women_script.iloc[i]['Character (actual)'] != women_script.iloc[i+1]['Character (actual)'] and \
    women_script.iloc[i+1]['Character (actual)'] in women and \
    women_script.iloc[i+1]['Line number'] - women_script.iloc[i]['Line number'] == 1:
        women_indices.append(i)
        women_indices.append(i+1)
women_indices.append(len(women_script)-1)

result = women_script.iloc[women_indices][['Character (actual)', 'Line', 'Place']]


In [152]:
result

,Character (actual),Line,Place
11,Waitress,Can I get anybody anymore coffee?,int. coffee shop
12,Honey Bunny,"Oh yes, thank you.",int. coffee shop
32,Waitress,"""Garcon"" means boy.",int. coffee shop
33,Honey Bunny,Here? It's a coffee shop.,int. coffee shop
251,Jody,...I'll lend it to you. It's a great book on b...,int. Lance's house (kitchen)
252,Trudi,You know how they use that gun when they pierc...,int. Lance's house (kitchen)
252,Trudi,You know how they use that gun when they pierc...,int. Lance's house (kitchen)
253,Jody,Forget that gun. That gun goes against the ent...,int. Lance's house (kitchen)
524,Mia,Something.,living room
525,Jody,Anybody want a beer?,living room


In [55]:
print(script.Line[251])
print(script.Line[252])
print(script.Line[253])

...I'll lend it to you. It's a great book on body piercing.
You know how they use that gun when they pierce your ears? They don't use that when they pierce your nipples, do they?
Forget that gun. That gun goes against the entire idea behind piercing. All of my piercing, sixteen places on my body, every one of 'em done with a needle. Five in each ear. One through the nipple on my left breast. One through my right nostril. One through my left eyebrow. One through my lip. One in my clit. And I wear a stud in my tongue.


In [57]:
pattern = r"\b(foot|feet|shoe|sock|toe)s?\b"
regex = re.compile(pattern, flags=re.IGNORECASE)
counts = script["Line"].str.count(regex).sum()

print(f"Total occurrences: {counts}")

Total occurrences: 24


In [59]:
pattern2 = r"\b(Red Apple)s?\b"
regex2 = re.compile(pattern2, flags=re.IGNORECASE)
counts2 = script["Line"].str.count(regex2).sum()

print(f"Total occurrences: {counts2}")

Total occurrences: 2


In [64]:
pattern3 = r"\b(Kahuna)s?\b"

regex3 = re.compile(pattern3, flags=re.IGNORECASE)
counts3 = script["Line"].str.count(regex3).sum()

print(f"Total occurrences: {counts3}")

Total occurrences: 3


In [63]:
mask = script["Line"].str.contains(regex)
matched_rows = script[mask]
matched_rows

C:\Users\gqsal\AppData\Local\Temp\ipykernel_4020\3561032519.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = script["Line"].str.contains(regex)


,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count
108,109,Jules,Jules,False,False,int. elevator,morning,He gave her a foot massage.,6
109,110,Vincent,Vincent,False,False,int. elevator,morning,A foot massage?,3
116,117,Vincent,Vincent,False,False,int. apartment building hallway,morning,You don't be givin' Marsellus Wallace's new br...,11
119,120,Jules,Jules,False,False,int. apartment building hallway,morning,"It was a foot massage, a foot massage is nothi...",17
121,122,Jules,Jules,False,False,int. apartment building hallway,morning,Whoa... whoa... whoa... stop right there. Eati...,23
123,124,Jules,Jules,False,False,int. apartment building hallway,morning,It ain't no ballpark either. Look maybe your m...,47
124,125,Vincent,Vincent,False,False,int. apartment building hallway,morning,Have you ever given a foot massage?,7
125,126,Jules,Jules,False,False,int. apartment building hallway,morning,Don't be tellin' me about foot massages – I'm ...,12
128,129,Vincent,Vincent,False,False,int. apartment building hallway,morning,Have you ever given a guy a foot massage?,9
132,133,Vincent,Vincent,False,False,int. apartment building hallway,morning,Would you give me a foot massage – I'm kinda t...,10


In [65]:
mask2 = script["Line"].str.contains(regex2)
matched_rows2 = script[mask2]
matched_rows2

C:\Users\gqsal\AppData\Local\Temp\ipykernel_4020\4026107325.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask2 = script["Line"].str.contains(regex2)


,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count
242,243,Butch,Butch,False,False,int. Sally LeRoy's,day,Can I get a pack'a Red Apples?,7
250,251,English Dave,English Dave,True,False,int. Sally LeRoy's,day,"Pack of Red Apples, dollar-forty.",5


In [66]:
mask3 = script["Line"].str.contains(regex3)
matched_rows3 = script[mask3]
matched_rows3

C:\Users\gqsal\AppData\Local\Temp\ipykernel_4020\346753647.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask3 = script["Line"].str.contains(regex3)


,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count
163,164,Brett,Brett,False,False,int. apartment (room 49),morning,Big Kahuna Burger.,3
164,165,Jules,Jules,False,False,int. apartment (room 49),morning,Big Kahuna Burger. That's that Hawaiian burger...,24
170,171,Jules,Jules,False,False,int. apartment (room 49),morning,"Uuummmm, that's a tasty burger. Vince, you eve...",13


In [147]:
import nltk
from nltk.corpus import stopwords
from collections import Counter

nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

def extract_words(text):
    words = nltk.word_tokenize(text)
    words = [word.lower() for word in words if word.lower() not in stop_words and len(word) > 2]
    return words

script['Words'] = script['Line'].apply(extract_words)
all_words = [word for words in script['Words'] for word in words]
word_counts = Counter(all_words)
most_common = word_counts.most_common(50)
most_common

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gqsal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gqsal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[("n't", 266),
 ("'re", 123),
 ('fuckin', 91),
 ('got', 85),
 ('gon', 77),
 ('like', 75),
 ('know', 72),
 ('...', 69),
 ('get', 68),
 ('one', 67),
 ('shit', 65),
 ("'ll", 62),
 ('man', 61),
 ('well', 48),
 ('good', 42),
 ('want', 39),
 ('right', 35),
 ('marsellus', 35),
 ('never', 34),
 ('say', 33),
 ('take', 33),
 ('tell', 33),
 ("'em", 33),
 ('fuck', 33),
 ('think', 33),
 ('ass', 33),
 ('give', 32),
 ('let', 32),
 ('would', 29),
 ('two', 27),
 ('mean', 26),
 ('little', 26),
 ('wan', 26),
 ('time', 25),
 ("'ve", 25),
 ('cool', 25),
 ('okay', 25),
 ('watch', 25),
 ('way', 24),
 ('car', 24),
 ('jimmie', 24),
 ('thing', 23),
 ('yeah', 23),
 ('need', 22),
 ('dead', 22),
 ('back', 22),
 ('big', 21),
 ('ever', 21),
 ('vincent', 21),
 ('yes', 20)]

In [134]:
grouped = script.groupby("Character (actual)").agg(
    Count=("Word count", "count"),
    Sum=("Word count", "sum"),
    Sentiment=("sentiment", "mean")
)

sorted_df = grouped.sort_values("Count", ascending=False)

sorted_df.index.name = "Character"
importance = sorted_df.reset_index()
print(importance)

                 Character  Count   Sum  Sentiment
0                  Vincent    259  2586   0.027086
1                    Jules    209  3221  -0.019805
2                    Butch    135  1254   0.070944
3                 Fabienne     90   705   0.060812
4                      Mia     81  1009   0.016401
5                  Pumpkin     62   881   0.040687
6                     Wolf     54  1117   0.161430
7                    Lance     50   693   0.018406
8              Honey Bunny     39   267   0.006649
9                    Brett     27    89   0.117244
10                  Jimmie     26   397  -0.022785
11                 Maynard     21   157  -0.038476
12                    Jody     21   212  -0.135557
13               Esmarelda     20   153  -0.147295
14               Marsellus     20   453  -0.212695
15            English Dave     11    84  -0.029055
16                     Zed     10    67  -0.020420
17             Ed Sullivan      7    77   0.126386
18                   Buddy     

In [136]:
mentions = []
for character in importance['Character']:
    pattern = re.escape(character)
    mentions.append(script['Line'].str.contains(pattern, case=False).sum())

importance['Mentions'] = mentions

print(importance)

                 Character  Count   Sum  Sentiment  Mentions
0                  Vincent    259  2586   0.027086        22
1                    Jules    209  3221  -0.019805        18
2                    Butch    135  1254   0.070944        18
3                 Fabienne     90   705   0.060812         5
4                      Mia     81  1009   0.016401        12
5                  Pumpkin     62   881   0.040687         2
6                     Wolf     54  1117   0.161430         8
7                    Lance     50   693   0.018406         3
8              Honey Bunny     39   267   0.006649         3
9                    Brett     27    89   0.117244         6
10                  Jimmie     26   397  -0.022785        22
11                 Maynard     21   157  -0.038476         1
12                    Jody     21   212  -0.135557         1
13               Esmarelda     20   153  -0.147295         4
14               Marsellus     20   453  -0.212695        31
15            English Da

In [137]:
imp = importance[importance['Count'] >= 20][['Character', 'Count', 'Sum', 'Mentions', "Sentiment"]]
imp

,Character,Count,Sum,Mentions,Sentiment
0,Vincent,259,2586,22,0.027086
1,Jules,209,3221,18,-0.019805
2,Butch,135,1254,18,0.070944
3,Fabienne,90,705,5,0.060812
4,Mia,81,1009,12,0.016401
5,Pumpkin,62,881,2,0.040687
6,Wolf,54,1117,8,0.161430
7,Lance,50,693,3,0.018406
8,Honey Bunny,39,267,3,0.006649
9,Brett,27,89,6,0.117244


In [162]:
imp["Quality"]=imp["Sum"]/imp["Count"]
imp["Mentions Ratio"]=imp["Mentions"]/imp["Count"]
imp

,Character,Count,Sum,Mentions,Sentiment,Quality,Mentions Ratio
0,Vincent,259,2586,22,0.027086,9.984556,0.084942
1,Jules,209,3221,18,-0.019805,15.411483,0.086124
2,Butch,135,1254,18,0.070944,9.288889,0.133333
3,Fabienne,90,705,5,0.060812,7.833333,0.055556
4,Mia,81,1009,12,0.016401,12.456790,0.148148
5,Pumpkin,62,881,2,0.040687,14.209677,0.032258
6,Wolf,54,1117,8,0.161430,20.685185,0.148148
7,Lance,50,693,3,0.018406,13.860000,0.060000
8,Honey Bunny,39,267,3,0.006649,6.846154,0.076923
9,Brett,27,89,6,0.117244,3.296296,0.222222


In [164]:
imp.to_csv("character.csv", index=False)

In [138]:
script["sentiment"] = script.Line.apply(lambda x: sia.polarity_scores(x)["compound"])

In [142]:
script.sample()

,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count,sentiment
934,935,Marsellus,Marsellus,False,False,int. Marsellus Wallace's dining room,morning,"...well, say she comes home. Whaddya think she...",31,0.6189


In [165]:
time=script.groupby("Time").agg(sentiment=("sentiment", "mean"))
time

,sentiment
Time,
day,-0.026534
morning,0.028143
night,0.014163


In [166]:
time.to_csv("time.csv")

In [154]:
place=script.groupby("Place").agg(sentiment=("sentiment", "mean")).sort_values(by="sentiment")
place

,sentiment
Place,
int. garage,-0.652200
int. Butch's apartment,-0.557400
ext. Sally LeRoy's,-0.542300
int. Jackrabbit Slim's (ladies room),-0.525500
ext. Lance's house,-0.358600
int. reception area (apartment building),-0.333180
int. Jimmie's kitchen,-0.332606
ext. alley (raining),-0.272950
int. taxi (parked / raining),-0.209033


In [167]:
place.to_csv("Place.csv")

In [156]:
script["Gender"] = np.where(script["Character (actual)"].isin(women), "W", "M")
script.sample()

,Line number,Character (in script),Character (actual),Off screen,Voice-over,Place,Time,Line,Word count,sentiment,Words,Gender
970,971,Jimmie,Jimmie,False,False,int. kitchen,morning,Yeah. Exactly. Under the sink.,5,0.296,"[yeah, exactly, sink]",M


In [157]:
gender=script.groupby("Gender").agg(sentiment=("sentiment", "mean"))
gender

,sentiment
Gender,
M,0.019689
W,0.011587


In [169]:
gender.to_csv("gender.csv")